In [1]:
import numpy as np
import torch
from smplx import SMPLX
from os import path as osp
import copy
import mujoco
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../utils")
from mujoco_parser import MuJoCoParserClass
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

MuJoCo version:[3.1.6]


In [2]:
motion_dir = '../data/idea400/smplx'
label_dir = '../data/idea400/label'
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Computation device: {}'.format(comp_device))

file_name = 'Arm_Swing_during_walking_1_clip1'
motion_npy = osp.join(motion_dir,file_name+'.npy')
mdata = np.load(motion_npy)

label_txt = osp.join(label_dir,file_name+'.txt')
with open(label_txt,'r') as f:
    full_annot = f.read()

seq_len = mdata.shape[0]
motion = torch.tensor(mdata).float()
motion_parms = {
            'root_orient': motion[:, :3].to(comp_device),  # controls the global root orientation
            'pose_body': motion[:, 3:3+63].to(comp_device),  # controls the body
            'pose_hand': motion[:, 66:66+90].to(comp_device),  # controls the finger articulation
            'trans': motion[:, 309:309+3].to(comp_device),  # controls the global body position
            'betas': motion[:, 312:].to(comp_device),  # controls the body shape. Body shape is static
        }

print('Body parameter vector shapes: \n{}'.format(' \n'.join(['{}: {}'.format(k,v.shape) for k,v in motion_parms.items()])))

Computation device: cpu
Body parameter vector shapes: 
root_orient: torch.Size([229, 3]) 
pose_body: torch.Size([229, 63]) 
pose_hand: torch.Size([229, 90]) 
trans: torch.Size([229, 3]) 
betas: torch.Size([229, 10])


In [3]:
betas = motion_parms['betas']
body_pose = motion_parms['pose_body']
left_hand_pose = motion_parms['pose_hand'][:, :45]
right_hand_pose = motion_parms['pose_hand'][:, 45:]

model = SMPLX(model_path='../models/smplx_models/male/model.npz', gender='male', num_betas=10, use_pca=False, flat_hand_mean=False).to(comp_device)
joints = []
for i in range(seq_len):
    output = model(
        betas=betas[i].unsqueeze(0),
        body_pose=body_pose[i].unsqueeze(0),
        root_orient=torch.tensor([0.0, 0.0, 0.0]).unsqueeze(0),
        transl=torch.tensor([0.0, 0.0, 0.0]).unsqueeze(0)
    )
    joints.append(output.joints.detach().cpu().numpy().squeeze())
joints = np.array(joints)

body_hand_index = list(range(0,22)) + list(range(25,55))
array = joints[:,body_hand_index,:]
array = array[:,:,[2,0,1]]
array -= array[0,0,:]
array += [0,0,0.98]
joints = array
print(joints.shape)

(229, 52, 3)


In [4]:
import mujoco
import numpy as np
import matplotlib.pyplot as plt
from mujoco_parser import MuJoCoParserClass
from util import rpy2r_order,get_idxs
np.set_printoptions(precision=2,suppress=True,linewidth=100)
plt.rc('xtick',labelsize=6); plt.rc('ytick',labelsize=6)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("MuJoCo version:[%s]"%(mujoco.__version__))

xml_path = '../asset/smplh_rig/scene_smplh_rig.xml'
env = MuJoCoParserClass(name='Common-Rig',rel_xml_path=xml_path,VERBOSE=True)

# Modify the color of body exluding 'world'
for body_name in env.body_names:
    if body_name in ['world']: continue
    for geom_idx in env.get_geom_idxs_from_body_name(body_name=body_name):
        env.model.geom(geom_idx).rgba = [0.3,0.3,0.5,0.5]

SMPLH_BODY_NAMES = ['pelvis','left_hip','right_hip','spine1','left_knee',
                    'right_knee','spine2','left_ankle','right_ankle','spine3','left_foot',
                    'right_foot','neck','left_collar','right_collar','head',
                    'left_shoulder','right_shoulder','left_elbow','right_elbow','left_wrist','right_wrist']
    
SMPLH_HAND_NAMES = ['left_index1', 'left_index2','left_index3',
                    'left_middle1','left_middle2','left_middle3',
                    'left_pinky1','left_pinky2','left_pinky3',
                    'left_ring1','left_ring2','left_ring3',
                    'left_thumb1','left_thumb2','left_thumb3',
                    'right_index1','right_index2','right_index3',
                    'right_middle1','right_middle2','right_middle3',
                    'right_pinky1','right_pinky2','right_pinky3',
                    'right_ring1','right_ring2','right_ring3',
                    'right_thumb1','right_thumb2','right_thumb3']

BODY_JOINT_NAMES = ['pelvis1', 'pelvis2', 'pelvis3', 'spine1x', 'spine1y', 'spine1z', 
                    'spine2x', 'spine2y', 'spine2z', 'spine3x', 'spine3y', 'spine3z','head1', 'head2', 'head3', 
                    'l_collar', 'l_shoulder1', 'l_shoulder2', 'l_shoulder3', 'l_elbow',  
                    'r_collar', 'r_shoulder1', 'r_shoulder2', 'r_shoulder3', 'r_elbow', 
                    'l_hip1', 'l_hip2', 'l_hip3', 'l_knee', 'l_ankle1', 'l_ankle2', 'l_ankle3', 
                    'r_hip1', 'r_hip2', 'r_hip3', 'r_knee', 'r_ankle1', 'r_ankle2', 'r_ankle3']

HAND_JOINT_NAMES = ['l_indexr', 'l_index1', 'l_index2', 'l_index3', 'l_middle1', 'l_middle2', 'l_middle3', 
                    'l_pinkyr', 'l_pinky1', 'l_pinky2', 'l_pinky3', 'l_ringr', 'l_ring1', 'l_ring2', 'l_ring3', 
                    'l_thumbr', 'l_thumba', 'l_thumb2', 'l_thumb3', 'l_wrist1', 'l_wrist2', 'l_wrist3',
                    'r_indexr', 'r_index1', 'r_index2', 'r_index3', 'r_middle1', 'r_middle2', 'r_middle3', 
                    'r_pinkyr', 'r_pinky1', 'r_pinky2', 'r_pinky3', 'r_ringr', 'r_ring1', 'r_ring2', 'r_ring3', 
                    'r_thumbr', 'r_thumba', 'r_thumb2', 'r_thumb3', 'r_wrist1', 'r_wrist2', 'r_wrist3']


MuJoCo version:[3.1.6]
dt:[0.0020] HZ:[500]
n_dof (=nv):[89]
n_geom:[73]
geom_names:['floor', 'base', 'base2lpelvis', 'base2rpelvis', 'pelvis2spine1', 'spine12spine2', 'spine22spine3', 'spine32neck', 'spine32lcollar', 'spine2rcollar', 'neck2head', 'head', 'nose', 'lcollar2lshoulder', 'lshoulder2lelbow', 'lelbow2lwrist', 'lwrist2lindex0', 'lwrist2lmiddle0', 'lwrist2lpinky0', 'lwrist2lring0', 'lwrist2lthumb1', 'lindex0-lindex1', 'lindex1-lindex2', 'lindex2-lindex3', 'lindex3-lindextip', 'lmiddle0-lmiddle1', 'lmiddle1-lmiddle2', 'lmiddle2-lmiddle3', 'lmiddl3-lmiddletip', 'lpinky0-lpinky1', 'lpinky1-lpinky2', 'lpinky2-lpinky3', 'lpinky3-lpinkytip', 'lring0-lring1', 'lring1-lring2', 'lring2-lring3', 'lring3-lringtip', 'lthumb1-lthumb2', 'lthumb2-lthumb3', 'lthumb3-lthumbtip', 'rcollar2rshoulder', 'rshoulder2relbow', 'relbow2rwrist', 'rwrist2rindex0', 'rwrist2rmiddle0', 'rwrist2rpinky0', 'rwrist2rring0', 'rwrist2rthumb1', 'rindex0-rindex1', 'rindex1-rindex2', 'rindex2-rindex3', 'rindex3-rind

In [5]:
import pickle, time

start = time.time()
qpos_data = np.zeros((seq_len, env.data.qpos.shape[0]))

env.init_viewer(viewer_title='SMPLH-Rig',viewer_width=1200,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=180,distance=2.6,elevation=-25,lookat=[0.0,0.0,0.83])

for frame_number in range(seq_len):
    p_target = {}
    for (i,n) in enumerate(SMPLH_BODY_NAMES + SMPLH_HAND_NAMES):
        p_target[n] = joints[frame_number,i,:]

    idxs = get_idxs(env.rev_joint_names,BODY_JOINT_NAMES)
    q_mins_body,q_maxs_body = env.rev_joint_mins[idxs],env.rev_joint_maxs[idxs]
    joint_idxs_fwd_body = env.get_idxs_fwd(joint_names=BODY_JOINT_NAMES)
    joint_idxs_jac_body = env.get_idxs_jac(joint_names=BODY_JOINT_NAMES)

    idxs = get_idxs(env.rev_joint_names,HAND_JOINT_NAMES)
    q_mins_hand,q_maxs_hand = env.rev_joint_mins[idxs],env.rev_joint_maxs[idxs]
    joint_idxs_fwd_hand = env.get_idxs_fwd(joint_names=HAND_JOINT_NAMES)
    joint_idxs_jac_hand = env.get_idxs_jac(joint_names=HAND_JOINT_NAMES)

    q_body = env.get_qpos_joints(BODY_JOINT_NAMES)
    body_diff = 1e3

    UPDATE_RIG_BASE = True
    if UPDATE_RIG_BASE:
        p_base = joints[frame_number,0,:]
        R_base = np.eye(3)
        env.set_p_root(root_name='base',p=p_base)
        env.set_R_root(root_name='base',R=R_base)
        env.forward()

    prev_q_pos = env.data.qpos.copy()
    q_body = env.get_qpos_joints(BODY_JOINT_NAMES)
    while body_diff > 1e-4:
        for ik_tick in range(2):
            J_list,ik_err_list = [],[]
            for ik_body_name in SMPLH_BODY_NAMES:
                ik_p_trgt = p_target[ik_body_name]
                ik_R_trgt = None
                IK_P = True
                IK_R = False
                J,ik_err = env.get_ik_ingredients(
                    body_name=ik_body_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
                    IK_P=IK_P,IK_R=IK_R)
                J_list.append(J)
                ik_err_list.append(ik_err)
            J_stack      = np.vstack(J_list)
            ik_err_stack = np.hstack(ik_err_list)
            dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
            q_body = q_body + dq[joint_idxs_jac_body]

            q_body = np.clip(q_body,q_mins_body,q_maxs_body)
            env.forward(q=q_body,joint_idxs=joint_idxs_fwd_body)
            
        q_hand = env.get_qpos_joints(HAND_JOINT_NAMES)
        for ik_tick in range(2): # ik loop
            J_list,ik_err_list = [],[]
            for ik_hand_name in SMPLH_HAND_NAMES:
                ik_p_trgt = p_target[ik_hand_name]
                ik_R_trgt = None
                IK_P = True
                IK_R = False
                J,ik_err = env.get_ik_ingredients(
                    body_name=ik_hand_name,p_trgt=ik_p_trgt,R_trgt=ik_R_trgt,
                    IK_P=IK_P,IK_R=IK_R)
                J_list.append(J)
                ik_err_list.append(ik_err)
            J_stack      = np.vstack(J_list)
            ik_err_stack = np.hstack(ik_err_list)
            dq = env.damped_ls(J_stack,ik_err_stack,stepsize=1,eps=1e-2,th=np.radians(1.0))
            q_hand = q_hand + dq[joint_idxs_jac_hand] # update

            q_hand = np.clip(q_hand,q_mins_hand,q_maxs_hand)
            env.forward(q=q_hand,joint_idxs=joint_idxs_fwd_hand)

            coupled_joint_idxs_list = [[47,48],[31,32,33],[34,35,36],[42,43,44],[38,39,40],[41,37],
                                       [74,75],[58,59,60],[61,62,63],[65,66,67],[69,70,71],[68,64]]

            coupled_joint_weights_list = [[1,1],[1,3,2],[1,3,2],[1,3,2],[1,3,2],[2,5],
                                          [1,1],[1,3,2],[1,3,2],[1,3,2],[1,3,2],[2,5]]
            
            q_couple = env.get_q_couple(
                q_raw=env.data.qpos,
                coupled_joint_idxs_list=coupled_joint_idxs_list,
                coupled_joint_weights_list=coupled_joint_weights_list)
            q_hand = q_couple[joint_idxs_fwd_hand]
            q_hand = np.clip(q_hand,q_mins_hand,q_maxs_hand)

            q_env = env.data.qpos.copy()
            body_diff = np.linalg.norm(q_env - prev_q_pos)
            env.forward(q=q_hand,joint_idxs=joint_idxs_fwd_hand)
            prev_q_pos=q_env.copy()

    qpos_data[frame_number,:] = q_env

    if frame_number % 10 == 0:
        print("Frame %d/%d, Time taken: %.2fsec" % (frame_number, seq_len, time.time()-start))

    env.plot_T(p=np.zeros(3),R=np.eye(3),label='[%d/%d]'%(frame_number,seq_len))
    env.render()

env.close_viewer()

pkl_data = {
    'full_annot': full_annot,
    'length': seq_len,
    'qpos': qpos_data,
}
with open('%s.pkl' % file_name, 'wb') as f:
    pickle.dump(pkl_data, f)

seconds = time.time()-start
print("%s rigging completed. Total time taken: %dmin %dsec" % (file_name,seconds//60,seconds%60))

Frame 0/229, Time taken: 1.67sec
Frame 10/229, Time taken: 3.06sec
Frame 20/229, Time taken: 3.95sec
Frame 30/229, Time taken: 5.27sec
Frame 40/229, Time taken: 6.46sec
Frame 50/229, Time taken: 7.72sec
Frame 60/229, Time taken: 9.32sec
Frame 70/229, Time taken: 11.64sec
Frame 80/229, Time taken: 13.45sec
Frame 90/229, Time taken: 14.69sec
Frame 100/229, Time taken: 16.46sec
Frame 110/229, Time taken: 18.04sec
Frame 120/229, Time taken: 20.00sec
Frame 130/229, Time taken: 21.15sec
Frame 140/229, Time taken: 23.29sec
Frame 150/229, Time taken: 24.70sec
Frame 160/229, Time taken: 26.10sec
Frame 170/229, Time taken: 27.49sec
Frame 180/229, Time taken: 28.91sec
Frame 190/229, Time taken: 31.20sec
Frame 200/229, Time taken: 32.47sec
Frame 210/229, Time taken: 34.39sec
Frame 220/229, Time taken: 36.32sec
Arm_Swing_during_walking_1_clip1 rigging completed. Total time taken: 0min 37sec


In [8]:
import pickle

SAVE_VIDEO = True

video = []
tick = 0
# file_name = "../data/idea400_rigged/Arm_Swing_during_walking_1_clip1"
file_name = "./Arm_Swing_during_walking_1_clip1"
# file_name = './' + file_name
with open('%s.pkl' % file_name, 'rb') as f:
    data = pickle.load(f)

qpos_data = data['qpos']
seq_len = data['length']

# Initialize MuJoCo viewer
env.init_viewer(viewer_title='SMPLH-Rig',viewer_width=800,viewer_height=800,
                viewer_hide_menus=True)
env.update_viewer(azimuth=180,distance=3.0,elevation=-20,lookat=[0.02,-0.03,1.2])
env.reset()

while tick < seq_len:
    q = qpos_data[tick,:]
    
    env.forward(q=q,INCREASE_TICK=True)

    if env.loop_every(tick_every=1):
        # Plot world frame
        env.plot_T(p=np.zeros(3),R=np.eye(3,3),
                   PLOT_AXIS=True,axis_len=0.5,axis_width=0.005)
        env.plot_T(p=np.array([0,0,0.5]),R=np.eye(3,3),
                   PLOT_AXIS=False,label="tick:[%d]"%(tick))
        env.render()  
        frame = env.grab_image()
        video.append(frame)

        for rev_joint_idx,rev_joint_name in zip(env.rev_joint_idxs,env.rev_joint_names):
            axis_joint = env.model.jnt_axis[rev_joint_idx]
            p_joint,R_joint = env.get_pR_joint(joint_name=rev_joint_name)
            axis_world = R_joint@axis_joint
            axis_rgba = np.append(np.eye(3)[:,np.argmax(axis_joint)],0.2)
            axis_len,axis_r = 0.02,0.002
            env.plot_arrow_fr2to(
                p_fr=p_joint,p_to=p_joint+axis_len*axis_world,
                r=axis_r,rgba=axis_rgba)

    tick = tick + 1
    if tick == seq_len:
        if not SAVE_VIDEO: tick = 0

env.close_viewer()

if SAVE_VIDEO:
    import mediapy as media
    media.show_video(images=video,fps=30,width=700)

This browser does not support the video tag.
